<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/digit_recognizer_20200415.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!kaggle competitions download -c digit-recognizer

Download 100%.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
cp: cannot create regular file '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, s

In [3]:
# ▼
# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c digit-recognizer
# ▲
# ▼
!ls
# 解凍
!unzip test.csv.zip
!unzip train.csv.zip
!ls
# ▼
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# とりあえずファイルを読み込む
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
train.info()
# ▲
# ▼
!pip install -q keras

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau  
from sklearn.model_selection import train_test_split
# ▲

 55% 5.00M/9.16M [00:00<00:00, 45.4MB/s]
100% 9.16M/9.16M [00:00<00:00, 58.6MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 75.4MB/s]
  0% 0.00/6.09M [00:00<?, ?B/s]
100% 6.09M/6.09M [00:00<00:00, 97.6MB/s]
adc.json  sample_data		 test.csv.zip
drive	  sample_submission.csv  train.csv.zip
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
adc.json  sample_data		 test.csv      train.csv
drive	  sample_submission.csv  test.csv.zip  train.csv.zip


In [14]:
train_x = train.iloc[:, 1:]
# 【iloc】切り出し。iloc[行, 列]
train_x = train.iloc[:, 1:].values
# values
train_x = train.iloc[:, 1:].values.astype('float32')
# float32に変換
# float16
# 3桁
# float32
# 6桁
# float64
# 15桁
train_y = train.iloc[:, 0]
train_y = train.iloc[:, 0].values.astype('int32')
test_x = test.values.astype('float32')
print('train_x', train_x.shape)
print('train_y', train_x.shape)
print('test_x', test_x.shape)
# print('# 書き方')
# print('o train_x', train_x.shape)
# print('x train_x'+ str(train_x.shape))

train_x (42000, 784)
train_y (42000, 784)
test_x (28000, 784)


# memo
train_x (42000, 784)
784pxが一次元に並んでいる状態。

